In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# === Параметры объекта ===
omega_n = 1.0     # собственная частота
zeta = 0.0        # коэффициент демпфирования
K = 1.0            # коэффициент передачи

# === Настройки симуляции ===
dt = 0.001          # шаг дискретизации
T = 10.0           # длительность симуляции
t = np.arange(0, T, dt)

def simulate_pid(Kp, Ki, Kd):
    y = np.zeros_like(t)
    y_dot = np.zeros_like(t)
    e_int = 0.0
    prev_e = 0.0

    r = np.ones_like(t)

    for i in range(1, len(t)):
        e = r[i] - y[i-1]

        e_int += e * dt
        e_der = (e - prev_e) / dt
        prev_e = e

        u = Kp * e + Ki * e_int + Kd * e_der

        y_ddot = K * u - 2*zeta*omega_n*y_dot[i-1] - (omega_n**2)*y[i-1]
        y_dot[i] = y_dot[i-1] + y_ddot * dt
        y[i] = y[i-1] + y_dot[i] * dt

    plt.figure(figsize=(8,5))
    plt.plot(t, r, 'k--', label='Задание r(t)')
    plt.plot(t, y, 'b', label='Выход y(t)')
    plt.title(f"ПИД-регулятор: Kp={Kp}, Ki={Ki}, Kd={Kd}")
    plt.xlabel("Время, с")
    plt.ylabel("Выход y(t)")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(
    simulate_pid,
    Kp=FloatSlider(value=3.0, min=0.0, max=50.0, step=0.1, description='Kp'),
    Ki=FloatSlider(value=0.0, min=0.0, max=5.0, step=0.1, description='Ki'),
    Kd=FloatSlider(value=0.0, min=0.0, max=20.0, step=0.05, description='Kd')
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# === Параметры объекта ===
K = 1.0       # коэффициент передачи
T = 1.5       # постоянная времени
L = 0.5       # задержка, сек

# === Настройки симуляции ===
dt = 0.001     # шаг дискретизации
T_sim = 10.0  # длительность симуляции
t = np.arange(0, T_sim, dt)
delay_steps = int(L / dt)  # дискретное запаздывание

def simulate_pid(Kp, Ki, Kd, show_plot=True):
    y = np.zeros_like(t)
    u = np.zeros_like(t)
    e_int = 0.0
    prev_e = 0.0

    u_buffer = np.zeros(delay_steps + 1)

    r = np.ones_like(t)

    for k in range(1, len(t)):
        e = r[k] - y[k-1]
        e_int += e * dt
        e_der = (e - prev_e) / dt
        prev_e = e

        u_current = Kp * e + Ki * e_int + Kd * e_der

        u_buffer = np.roll(u_buffer, -1)
        u_buffer[-1] = u_current
        u_delayed = u_buffer[0]  

        y[k] = y[k-1] + dt / T * (-y[k-1] + K * u_delayed)
        u[k] = u_current

    if show_plot:
        plt.figure(figsize=(8,5))
        plt.plot(t, r, 'k--', label='Задание r(t)')
        plt.plot(t, y, 'b', label='Выход y(t)')
        # plt.plot(t, u, 'r', label='Управление u(t)', alpha=0.5)
        plt.title(f"ПИД-регулятор: Kp={Kp:.2f}, Ki={Ki:.2f}, Kd={Kd:.2f}, L={L}s")
        plt.xlabel("Время, с")
        plt.ylabel("Выход / Управление")
        plt.grid(True)
        plt.legend()
        plt.show()

    return y

interact(
    simulate_pid,
    Kp=FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description='Kp'),
    Ki=FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description='Ki'),
    Kd=FloatSlider(value=0.2, min=0.0, max=2.0, step=0.05, description='Kd'),
    show_plot=True
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, HBox, VBox, Output

from scipy.signal import find_peaks, welch

# === Параметры объекта ===
K = 1.0       # коэффициент передачи
T = 1.5       # постоянная времени
L = 0.5       # задержка, сек

# === Настройки симуляции ===
dt = 0.001     # шаг дискретизации
T_sim = 10.0  # длительность симуляции
t = np.arange(0, T_sim, dt)
delay_steps = int(L / dt)  # дискретное запаздывание

def simulate_pid(Kp, Ki, Kd, show_plot=True):
    y = np.zeros_like(t)
    u = np.zeros_like(t)
    e_int = 0.0
    prev_e = 0.0

    u_buffer = np.zeros(delay_steps + 1)

    r = np.ones_like(t)

    for k in range(1, len(t)):
        e = r[k] - y[k-1]
        e_int += e * dt
        e_der = (e - prev_e) / dt
        prev_e = e

        u_current = Kp * e + Ki * e_int + Kd * e_der

        u_buffer = np.roll(u_buffer, -1)
        u_buffer[-1] = u_current
        u_delayed = u_buffer[0] 

        y[k] = y[k-1] + dt / T * (-y[k-1] + K * u_delayed)
        u[k] = u_current

    if show_plot:
        plt.figure(figsize=(8,5))
        plt.plot(t, r, 'k--', label='Задание r(t)')
        plt.plot(t, y, 'b', label='Выход y(t)')
        # plt.plot(t, u, 'r', label='Управление u(t)', alpha=0.5)
        plt.title(f"ПИД-регулятор: Kp={Kp:.2f}, Ki={Ki:.2f}, Kd={Kd:.2f}, L={L}s")
        plt.xlabel("Время, с")
        plt.ylabel("Выход / Управление")
        plt.grid(True)
        plt.legend()
        plt.show()

    return y

def auto_tune(show_plot=True):
    Ku = None
    Tu = None

    # Итерируем коэффициент усиления до начала самоколебаний
    for Kp_test in np.linspace(0.1, 20, 1000):
        y = simulate_pid(Kp_test, 0, 0, show_plot=False)
        steady = y[int(0.7 * len(y)):]  # последние 30% данных
        osc = steady - np.mean(steady)

        # Проверяем, появились ли устойчивые колебания
        if np.max(osc) - np.min(osc) > 0.5 and np.std(osc) > 0.1:
            Ku = Kp_test

            # Нахождения периода ===
            f, Pxx = welch(osc, fs=1/dt)
            f_dom = f[np.argmax(Pxx[1:]) + 1]  # доминирующая частота
            Tu = 1 / f_dom if f_dom > 0 else None
            break

    if Ku is None or Tu is None:
        print("Не удалось найти устойчивые колебания.")
        return None

    # === Формулы Зиглера–Николса ===
    Kp = 0.6 * Ku
    Ki = 1.2 * Kp / Tu
    Kd = 0.075 * Kp * Tu

    print(f"Найдено Ku={Ku:.2f}, Tu={Tu:.2f}")
    print(f"Рекомендуемые параметры PID:")
    print(f"Kp = {Kp:.2f}, Ki = {Ki:.2f}, Kd = {Kd:.2f}")

    if show_plot:
        simulate_pid(Kp, Ki, Kd)

    return Kp, Ki, Kd

def interactive_pid(Kp, Ki, Kd):
    simulate_pid(Kp, Ki, Kd)

interact(
    interactive_pid,
    Kp=FloatSlider(value=3.0, min=0.0, max=100.0, step=0.1, description='Kp'),
    Ki=FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description='Ki'),
    Kd=FloatSlider(value=0.5, min=0.0, max=2.0, step=0.05, description='Kd')
);

print("Нажми для подбора коэффициентов по методу Зиглера–Николса:")
btn = Button(description="🔧 Автонастройка PID")
out = Output()

def on_button_clicked(_):
    with out:
        out.clear_output()
        auto_tune()

btn.on_click(on_button_clicked)
display(VBox([btn, out]))

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

class DeltaPIDNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 16),
            nn.Tanh(),
            nn.Linear(16, 8),
            nn.Tanh(),
            nn.Linear(8, 3)
        )

    def forward(self, e, e_dot, e_int):
        x = torch.stack([e, e_dot, e_int], dim=-1)
        return self.net(x)

def system_step(y, y_dot, u, dt, omega_n=2.0, zeta=0.3, K=1.0):
    y_ddot = K*u - 2*zeta*omega_n*y_dot - (omega_n**2)*y
    y_dot_next = y_dot + y_ddot * dt
    y_next = y + y_dot_next * dt
    return y_next, y_dot_next

dt = 0.01
T = 1000.0
steps = int(T / dt)
r = 1.0

model = DeltaPIDNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Kp, Ki, Kd = 1.0, 0.0, 0.0 
y, y_dot = 0.0, 0.0
e_int, prev_e = 0.0, 0.0

losses = []
y_hist, t_hist = [], []
Kp_hist, Ki_hist, Kd_hist = [], [], []

for step in range(steps):
    t = step * dt
    e = r - y
    e_dot = (e - prev_e) / dt
    e_int += e * dt
    prev_e = e

    e_t = torch.tensor([e, e_dot, e_int], dtype=torch.float32)
    dKp, dKi, dKd = model(e_t[0], e_t[1], e_t[2])

    Kp += 0.01 * dKp.item()
    Ki += 0.01 * dKi.item()
    Kd += 0.01 * dKd.item()

    Kp = np.clip(Kp, 0.0, 10.0)
    Ki = np.clip(Ki, 0.0, 5.0)
    Kd = np.clip(Kd, 0.0, 2.0)

    u = Kp*e + Ki*e_int + Kd*e_dot
    u = np.clip(u, -10, 10)

    y, y_dot = system_step(y, y_dot, u, dt)

    loss = torch.abs(e_t[0]).requires_grad_()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    y_hist.append(y)
    t_hist.append(t)
    Kp_hist.append(Kp)
    Ki_hist.append(Ki)
    Kd_hist.append(Kd)

plt.figure(figsize=(8,5))
plt.plot(t_hist, np.ones_like(t_hist)*r, 'k--', label='Задание')
plt.plot(t_hist, y_hist, label='Выход системы')
plt.legend(); plt.grid(); plt.title("Neuro-PID с прогнозом ΔK")
plt.xlabel("Время, с")
plt.show()

plt.figure()
plt.plot(Kp_hist, label="Kp")
plt.plot(Ki_hist, label="Ki")
plt.plot(Kd_hist, label="Kd")
plt.title("Динамика коэффициентов PID")
plt.grid(); plt.legend()
plt.show()

plt.figure()
plt.plot(losses)
plt.title("Ошибка (|e|) во времени")
plt.grid()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# === FOPDT ===
def fopdt_sim(pid, K=1.0, T=2.0, L=0.5, dt=0.01, Tsim=10.0, r_value=1.0):
    steps = int(Tsim / dt)
    delay_steps = int(L / dt)

    y = torch.zeros(steps)
    u = torch.zeros(steps)
    e_int = torch.tensor(0.0)
    prev_e = torch.tensor(0.0)

    for k in range(1, steps):
        if k - delay_steps >= 0:
            u_delayed = u[k - delay_steps]
        else:
            u_delayed = torch.tensor(0.0)

        y[k] = y[k-1] + (K * u_delayed - y[k-1]) * (dt / T)

        e = torch.tensor(r_value) - y[k]
        e_int = e_int + e * dt
        e_dot = (e - prev_e) / dt
        prev_e = e

        u[k] = pid(e, e_int, e_dot)
        u[k] = torch.clamp(u[k], -10.0, 10.0)

    r = torch.ones_like(y) * r_value
    loss = torch.mean(torch.abs(r - y))
    return loss, y.detach()

# === PID-контроллер с обучаемыми параметрами ===
class PIDController(nn.Module):
    def __init__(self):
        super().__init__()
        self.Kp_raw = nn.Parameter(torch.tensor(0.0))
        self.Ki_raw = nn.Parameter(torch.tensor(0.0))
        self.Kd_raw = nn.Parameter(torch.tensor(0.0))
        self.softplus = nn.Softplus()

    def forward(self, e, e_int, e_dot):
        Kp = self.softplus(self.Kp_raw)
        Ki = self.softplus(self.Ki_raw)
        Kd = self.softplus(self.Kd_raw)
        return Kp * e + Ki * e_int + Kd * e_dot

pid = PIDController()
optimizer = optim.Adam(pid.parameters(), lr=0.01)
losses = []

for epoch in range(200):
    optimizer.zero_grad()
    loss, y_hist = fopdt_sim(pid, K=1.0, T=2.0, L=0.5, dt=0.01, Tsim=10.0)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if epoch % 20 == 0:
        with torch.no_grad():
            Kp = torch.nn.functional.softplus(pid.Kp_raw).item()
            Ki = torch.nn.functional.softplus(pid.Ki_raw).item()
            Kd = torch.nn.functional.softplus(pid.Kd_raw).item()
        print(f"Epoch {epoch:3d} | Loss={loss.item():.4f} | Kp={Kp:.3f}, Ki={Ki:.3f}, Kd={Kd:.3f}")

with torch.no_grad():
    _, y_hist = fopdt_sim(pid, K=1.0, T=2.0, L=0.5, dt=0.01, Tsim=10.0)

plt.figure(figsize=(10,4))
plt.plot(y_hist.numpy(), label="Выход системы")
plt.plot(torch.ones_like(y_hist).numpy(), "k--", label="Задание")
plt.legend(); plt.grid(); plt.title("FOPDT с обученными PID")
plt.show()

plt.figure()
plt.plot(losses)
plt.title("Эволюция функции потерь")
plt.grid()
plt.show()